<a href="https://colab.research.google.com/github/chongzicbo/Dive-into-Deep-Learning-tf.keras/blob/master/4.1.%20%E6%A8%A1%E5%9E%8B%E6%9E%84%E9%80%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras

In [0]:
tf.enable_eager_execution()#启用计算图

###4.1.1 继承keras.layers.Layer来构建新的神经网络层
&emsp;&emsp;在keras中，构建新的神经网络层可以通过继承keras.layers.Layer类来实现。你只需要实现三个方法即可:

* build(input_shape): 这是你定义权重的地方。这个方法必须设 self.built = True，可以通过调用 super([Layer], self).build() 完成。
* call(x): 这里是编写层的功能逻辑的地方。你只需要关注传入 call 的第一个参数：输入张量，除非你希望你的层支持masking。
* compute_output_shape(input_shape): 如果你的层更改了输入张量的形状，你应该在这里定义形状变化的逻辑，这让Keras能够自动推断各层的形状。

&emsp;&emsp;这里我们通过集成两个Dense层来构造一个MLP网络层。

In [0]:
class MLP(keras.layers.Layer):
  def __init__(self):
    super(MLP,self).__init__()
    self.hidden=keras.layers.Dense(256,activation='relu') #隐藏层
    self.outputs=keras.layers.Dense(10) #输出层

  def build(self,input_shape):
    print(f"{self.name} build() is called.")
    self.build=True

  def call(self,inputs):
    print(f"{self.name} call() is called.")
    x=self.hidden(inputs)
    x=self.outputs(x)
    return x

&emsp;&emsp;构造完自己的神经网络层后，可以将其实例化后输入一个张量，并获得一个张量输出。

In [4]:
x=tf.random.normal(shape=(2,20))
mlp=MLP()(x)
mlp

mlp build() is called.
mlp call() is called.


<tf.Tensor: id=59, shape=(2, 10), dtype=float32, numpy=
array([[-0.30091882, -0.07629362, -0.71176815,  0.4058078 , -0.14221677,
        -0.04297486, -0.07796003, -0.23001648, -0.18110617, -0.54717225],
       [-0.2731866 , -0.1405192 , -0.2932847 ,  0.35041004, -0.2668861 ,
        -0.01136987, -0.15359104, -0.36639145,  0.15943211, -0.47985178]],
      dtype=float32)>

###4.1.2. 继承layers.Model类来构造模型
&emsp;&emsp;在keras中，除了Sequential顺序模型和使用函数式API的Model类模型这两类模型之外，你还可以通过继承Model类并在call方法中实现你自己的前向传播，以创建你自己的完全定制化的模型。这里是一个用 Model类继承写的简单的多层感知器的例子：

In [0]:
class MLPModel(keras.Model):
  def __init__(self):
    super(MLPModel,self).__init__()
    self.hidden=keras.layers.Dense(256,activation='relu') #隐藏层
    self.out=keras.layers.Dense(10) #输出层

  def call(self,inputs):
    return self.out(self.hidden(x)) #前向传播

In [0]:
net=MLPModel()

In [7]:
net.layers[0].name

'dense_2'

In [0]:
net.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),loss=keras.losses.CategoricalCrossentropy)

###4.1.3 keras.layers.Layer和keras.Model的区别
&emsp;&emsp;从它们的名称可以看出，Layer是一个层，而Model是一个模型；前者用于构造神经网络层，后者用于构建神经网络模型，可以说Layer由于构建Model，而Model包含各种Layer。一个层接受一个张量并给出一个张量，这是一些张量运算的结果而模型是多层的组合。\
&emsp;&emsp;如果使用现有的keras / tf层构建新的模型体系结构，则构建自定义模型。

如果要在图层中实现自己的自定义张量操作，则构建自定义图层。如果在自定义图层中使用非张量操作，则必须编码图层如何向前传播和向后传播。

###4.1.4. 小结
* 可以通过继承Layer类来构造自定义神经网络层。
* 可以通过继承Model类来构建神经网络模型。
* Layer是网络层，而Model是神经网络模型，两者有所不同。